# Specify Paths 

In [ ]:

fpga_v3_data_collection_path = "~/force_estimation/preprocess/fpgav3-data-collection/" # SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_path = fpga_v3_data_collection_path + "unit_convert/unit_convert.py"
si_unit_json_path = fpga_v3_data_collection_path + "unit_convert/sawRobotIO1394-PSM1-334809.xml.json"
unit_convert_cmd = "python3 " + unit_convert_path + " -c " + si_unit_json_path + " -f " 

train_name_path_csv = "/home/ndrakes1/force_estimation/data/capture_6-26-25/train.csv" # SET THIS
val_test_name_path_csv = "/home/ndrakes1/force_estimation/data/capture_6-26-25/val_split.csv" # SET THIS

train_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/capture_6-26-25/train_unitConvert.csv" # SET THIS
val_ttest_name_path_unit_convert_csv = "~/force_estimation/data/capture_6-26-25/val_split_unitConvert.csv" # SET THIS

path_to_data_foleder = "/home/ndrakes1/force_estimation/data/capture_6-26-25" ## SET THIS
new_preprocessed_data_folder = "/processed"

unit_convert_train_cmd = unit_convert_cmd + train_name_path_csv
unit_convert_val_test_cmd = unit_convert_cmd + val_test_name_path_csv

ORIGINAL_FREQ = 5000

CUTOFF = True
CUTOFF_SECS = 15

if CUTOFF == True:
    new_preprocessed_data_folder += "_CUTOFF_" + str(CUTOFF_SECS)

INTERPOLATE = True

if INTERPOLATE == True:
    new_preprocessed_data_folder += "_INTERPOLATE" 


FILTER = True
FILTER_FREQ = 200

if FILTER == True:
    new_preprocessed_data_folder += "_FILTER_" + str(FILTER_FREQ)

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 200
DOWNSAMPLE_MOVING_AVERAGE = False

if DOWNSAMPLE == True:
    new_preprocessed_data_folder += "_DOWNSAMPLE_" + str(DOWNSAMPLE_FREQ)



## paths for high freq preprocessing

In [47]:
preproces_path = "/home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess"
preprocessing_script_path = preproces_path + "/preprocessing.py"
split_val_test_path = preproces_path + "/split_val_test.py"

preprocessing_train_cmd = "python3 " + preprocessing_script_path + " " + train_name_path_unit_convert_csv
preprocessing_val_test_cmd ="python3 " + preprocessing_script_path + " " + val_ttest_name_path_unit_convert_csv

print(preprocessing_train_cmd)

python3 /home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess/preprocessing.py /home/ndrakes1/force_estimation/data/capture_6-26-25/train_unitConvert.csv


In [48]:
## making preprocessed data dir 
import subprocess

new_dir = path_to_data_foleder + new_preprocessed_data_folder
train_sub = new_dir + "/train/joints/"
val_sub = new_dir + "/val/joints/"
test_sub = new_dir + "/test/joints"
mk_new_dir_cmd = "mkdir -p " + train_sub + " " + val_sub + " " + test_sub

subprocess.run(mk_new_dir_cmd, shell=True)

CompletedProcess(args='mkdir -p /home/ndrakes1/force_estimation/data/capture_6-26-25/processed_CUTOFF_15_INTERPOLATE_FILTER_200_DOWNSAMPLE_200/train/joints/ /home/ndrakes1/force_estimation/data/capture_6-26-25/processed_CUTOFF_15_INTERPOLATE_FILTER_200_DOWNSAMPLE_200/val/joints/ /home/ndrakes1/force_estimation/data/capture_6-26-25/processed_CUTOFF_15_INTERPOLATE_FILTER_200_DOWNSAMPLE_200/test/joints', returncode=0)

# Unit Conversion to SI units

In [49]:

subprocess.run(unit_convert_val_test_cmd, shell=True)
subprocess.run(unit_convert_train_cmd, shell=True)

Data written to /home/ndrakes1/force_estimation/data/capture_6-26-25/val_split_unitConvert.csv
Data written to /home/ndrakes1/force_estimation/data/capture_6-26-25/train_unitConvert.csv


CompletedProcess(args='python3 ~/force_estimation/preprocess/fpgav3-data-collection/unit_convert/unit_convert.py -c ~/force_estimation/preprocess/fpgav3-data-collection/unit_convert/sawRobotIO1394-PSM1-334809.xml.json -f /home/ndrakes1/force_estimation/data/capture_6-26-25/train.csv', returncode=0)

# High Freq Preprocessing

In [50]:
## interpolate train and place it in correct directory
##train
subprocess.run(preprocessing_train_cmd, shell=True)
print("finished preprocessing train ")
interpolated_all_joints_path = preproces_path + "/interpolated_all_joints.csv"
mv_train_interpolated_cmd = "mv " + interpolated_all_joints_path + " " + train_sub
subprocess.run(mv_train_interpolated_cmd, shell=True)
print("finished moving train")

## val test
subprocess.run(preprocessing_val_test_cmd, shell=True)
print("preprocess split and val")

# # split
split_val_test_cmd = "python3 " + preproces_path + "/split_val_test.py " + interpolated_all_joints_path
subprocess.run(split_val_test_cmd, shell=True)
print("split into val and test")
#interpolate val test and split into val and test
val_path = val_sub
mv_val_cmd = "mv " + preproces_path + "/val.csv " + val_path
subprocess.run(mv_val_cmd, shell=True)
change_val_interpolated_cmd = "mv " + val_path + "/val.csv " + val_path + "/interpolated_all_joints.csv"
subprocess.run(change_val_interpolated_cmd, shell=True)

test_path = test_sub
mv_test_cmd = "mv " + preproces_path + "/test.csv " + test_path
subprocess.run(mv_test_cmd, shell=True)
change_test_interpolated_cmd = "mv " + test_path + "/test.csv " + test_path + "/interpolated_all_joints.csv"
subprocess.run(change_test_interpolated_cmd, shell=True)

finished preprocessing train 
finished moving train
preprocess split and val
Validation set saved to val.csv 406230 rows
Test set saved to test.csv 406231 rows
split into val and test


CompletedProcess(args='mv /home/ndrakes1/force_estimation/data/capture_6-26-25/processed_CUTOFF_15_INTERPOLATE_FILTER_200_DOWNSAMPLE_200/test/joints/test.csv /home/ndrakes1/force_estimation/data/capture_6-26-25/processed_CUTOFF_15_INTERPOLATE_FILTER_200_DOWNSAMPLE_200/test/joints/interpolated_all_joints.csv', returncode=0)

## Preprocessed Training File Paths

In [51]:
import os
import pandas as pd

train_file = os.path.join(train_sub, "interpolated_all_joints.csv")
val_file = os.path.join(val_sub, "interpolated_all_joints.csv")
test_file = os.path.join(test_sub, "interpolated_all_joints.csv")

train_df = pd.read_csv(train_file, header=None)
val_df =  pd.read_csv(val_file, header=None)
test_df = pd.read_csv(test_file, header=None)

## Cutoff


In [59]:
from cutoff import truncate_dataframe


if (CUTOFF == True):
    ## TRAIN
    train_df = truncate_dataframe(train_df, CUTOFF_SECS, ORIGINAL_FREQ)
    val_df = truncate_dataframe(val_df, CUTOFF_SECS, ORIGINAL_FREQ)
    test_df = truncate_dataframe(test_df, CUTOFF_SECS, ORIGINAL_FREQ)

## Interpolate

In [58]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):
    
    train_df = interpolate_dataframe_to_sample_rate(train_df, ORIGINAL_FREQ)
    val_df = interpolate_dataframe_to_sample_rate(val_df, ORIGINAL_FREQ)
    test_df = interpolate_dataframe_to_sample_rate(test_df, ORIGINAL_FREQ)


## FIR Filtering

In [55]:
import importlib
import filter  # regular import first
importlib.reload(filter)



if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to train/val/test CSVs
    train_df = filter.apply_filter_to_torque_feedback_df(train_df, fir_coeffs)
    val_df = filter.apply_filter_to_torque_feedback_df(val_df, fir_coeffs)
    test_df = filter.apply_filter_to_torque_feedback_df(test_df, fir_coeffs)

## Downsample


In [56]:
from downsample import downsample_dataframe

if (DOWNSAMPLE == True):
    train_df = downsample_dataframe(train_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    val_df = downsample_dataframe(val_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    test_df = downsample_dataframe(test_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


## SAVE TO CSV

In [57]:
train_df.to_csv(train_file, index=False, header=False)
val_df.to_csv(val_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)